<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#purpose" data-toc-modified-id="purpose-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>purpose</a></span></li><li><span><a href="#result-analysis-env" data-toc-modified-id="result-analysis-env-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>result analysis env</a></span></li><li><span><a href="#links" data-toc-modified-id="links-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>links</a></span></li></ul></div>

# purpose
This is a navigator notebook just for explanation and links to separate result demonstration notebooks

# result analysis env
python pkg for parameter simulation: scipy (1.4.1), numpy (1.18.2)  
Julia-lang check pkg version: using Pkg;Pkg.installed()  
RCall (0.13.4), DataFrames (0.20.2), CSV (0.6.1), Plots (0.29.7)  

In [1]:
using RCall, DataFrames, CSV, Plots
#rAw = CSV.read("../result/maxYield_0.1.csv")
rAw = CSV.read("../result/maxYield_0.05.csv")
describe(rAw)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Nothing,Nothing,DataType
1,x,0.466182,0.0,0.45,1.0,,,Float64
2,e_PR,0.563,0.563,0.563,0.563,,,Float64
3,e_P,0.63,0.63,0.63,0.63,,,Float64
4,g_P,0.41827,0.259,0.4373,0.556,,,Float64
5,a_P,0.184238,0.001,0.1606,0.4,,,Float64
6,e_BR,0.6,0.6,0.6,0.6,,,Float64
7,e_B,0.55,0.55,0.55,0.55,,,Float64
8,g_B,5.00411,1.046,5.1,8.414,,,Float64
9,m_B,0.14,0.14,0.14,0.14,,,Float64


# links
- [organic carbon](https://nbviewer.jupyter.org/github/ph-u/Project/blob/master/sandbox/cpb_orgC.ipynb)
- [photocell](https://nbviewer.jupyter.org/github/ph-u/Project/blob/master/sandbox/cpb_orgP.ipynb)
- [bacterial decomposer](https://nbviewer.jupyter.org/github/ph-u/Project/blob/master/sandbox/cpb_orgB.ipynb)
- [total carbon](https://nbviewer.jupyter.org/github/ph-u/Project/blob/master/sandbox/cpb_orgA.ipynb)